<a href="https://colab.research.google.com/github/AlexKressner/Vorlesung-Industrielles-Mangement/blob/main/%C3%9Cbungsaufgabe_Raffinerieproblem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Raffinerieproblem

In [73]:
# Notwendigen Programminstallationen
# pip als Paketmanager
!pip install -U -q pip
!pip install -q ortools
# Laden des Programms
from ortools.linear_solver import pywraplp

In [74]:
# Solver mit SCIP als Backend.
# SCIP implementiert Simplex, Branch-and-Bound, etc
solver = pywraplp.Solver.CreateSolver('SCIP')

## Indexmengen

In [75]:
J = ['K1','K2','K3','K4'] # Menge der Kraftstoffe
I = ['R1','R2','R3','Gesamtkapa'] # Menge der Produktionsressourcen

In [76]:
J

['K1', 'K2', 'K3', 'K4']

## Entscheidungsvariablen

In [77]:
infinity = solver.infinity()

In [78]:
# Definition der Entscheidungsvariablen
x={}
for j in J: 
    x[j] = solver.NumVar(0.0, infinity, j)
print('Anzahl Entscheidungsvariablen =', solver.NumVariables())

Anzahl Entscheidungsvariablen = 4


## Parameter

In [79]:
c={} # Deckungsbeiträge
c['K1']=5
c['K2']=8
c['K3']=15
c['K4']=3

In [80]:
a={} # Produktionskoeffizienten, a[ij]
a['R1']={'K1':1,'K2':1, 'K3':2,'K4':1,}
a['R2']={'K1':2,'K2':1, 'K3':3,'K4':0,}
a['R3']={'K1':0,'K2':1, 'K3':2,'K4':1,}
a['Gesamtkapa']={'K1':1,'K2':1, 'K3':1,'K4':1,}

In [81]:
b={} # Kapazitäten Produktionsfaktoren
b['R1']=1000
b['R2']=500
b['R3']=1500
b['Gesamtkapa']=2250

## Zielfunktion

In [82]:
# Maximierung der gesamten Deckungsbeiträge
solver.Maximize(sum(c[j]*x[j] for j in J))

## Nebenbedingungen

In [83]:
for i in I:
    solver.Add(sum(a[i][j]*x[j] for j in J)<=b[i])
print('Anzahl Nebenbedingungen =', solver.NumConstraints())

Anzahl Nebenbedingungen = 4


In [84]:
# Nebenbedingung Mindestkapazität Kraftstoff 1 (K1)
solver.Add(x["K1"]>=250)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7f7d4f261b40> >

## Berechnung Lösung

In [85]:
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('LÖSUNG:')
    print('Zielfunktionswert (Deckungsbeitrag) =', solver.Objective().Value())
    for j in J:
        print(f'{j} =', round(x[j].solution_value()))
else:
    print('Problem hat keine Lösung')

LÖSUNG:
Zielfunktionswert (Deckungsbeitrag) = 3500.0
K1 = 250
K2 = 0
K3 = 0
K4 = 750
